In [4]:
import pandas as pd
import numpy as np
import chart_studio.plotly as py
import plotly.graph_objs as pgo
import glob

In [15]:
tran_df = pd.read_excel("/Users/andres/Desktop/CSE498/appdynamics/training_csv/fam_3_edu_2_CCAvg_4_group.xlsx")
tran_df.head()

,id,Age,Experience,Income,ZIP.Code,Family,CCAvg,Education,Mortgage,Personal.Loan,Securities.Account,CD.Account,Online,CreditCard,responsetime,problem_group,portion,time
1,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0,1938.100070,6,0.073529,2020-02-15 00:00:00
2,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0,25.165190,0,0.000000,2020-02-15 00:02:52
3,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0,2267.739555,6,0.279412,2020-02-15 00:05:45
4,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0,18.932080,0,0.000000,2020-02-15 00:08:38
5,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1,25.627543,0,0.000000,2020-02-15 00:11:31


In [16]:
tran_df = pd.read_excel("/Users/andres/Desktop/CSE498/appdynamics/training_csv/fam_3_edu_2_CCAvg_4_group.xlsx")

problem_group_text = []
new_responsetime = []
portion = []
new_portion = []

problem_dict = {0 : "normal", 2 : "Family + Education + CCAvg", 6 : "All other anomalies"}

for index, row in tran_df.iterrows():
    problem_group_text.append(problem_dict[row['problem_group']])
    
tran_df['anomaly_class'] = problem_group_text

tran_df.head()

,id,Age,Experience,Income,ZIP.Code,Family,CCAvg,Education,Mortgage,Personal.Loan,Securities.Account,CD.Account,Online,CreditCard,responsetime,problem_group,portion,time,anomaly_class
1,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0,1938.100070,6,0.073529,2020-02-15 00:00:00,All other anomalies
2,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0,25.165190,0,0.000000,2020-02-15 00:02:52,normal
3,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0,2267.739555,6,0.279412,2020-02-15 00:05:45,All other anomalies
4,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0,18.932080,0,0.000000,2020-02-15 00:08:38,normal
5,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1,25.627543,0,0.000000,2020-02-15 00:11:31,normal


In [21]:
zipcode_to_latlong = pd.read_csv("./us-zip-code-latitude-and-longitude.csv")

In [22]:
zipcode_to_latlong.head()

,Zip,City,State,Latitude,Longitude,Timezone,Daylight savings time flag,geopoint
0,71937,Cove,AR,34.398483,-94.39398,-6,1,"34.398483, -94.39398"
1,72044,Edgemont,AR,35.624351,-92.16056,-6,1,"35.624351, -92.16056"
2,56171,Sherburn,MN,43.660847,-94.74357,-6,1,"43.660847, -94.74357"
3,49430,Lamont,MI,43.010337,-85.89754,-5,1,"43.010337, -85.89754"
4,52585,Richland,IA,41.194129,-91.98027,-6,1,"41.194129, -91.98027"


In [15]:
def data_maker(data=None, normal_noise=0.05, anomaly_noise=0.20, groups=3, feature_set=[['Mortgage', 'Family'], ['Education', 'Income'], ['ZIP.Code']],
               thresholds=[[[200, 1], [[3, 2], 4]], [[[1, 2], 4], [(50, 100), 3]], [[[91107], 4]]],
               anomaly_types=[2, 1, 3], feature_set_size=[2, 3, 4],
               random_groups=False, random_anomalies=False, random_features=False, random_thresholds=False,
               random_state=1):
    """
    params:
    data: pandas data frame
    normal_noise: portion of data that is set to an anomalous value regardless of thresholds
    anomaly_noise: portion of data that within anomalous thresholds that performs normaly
    groups: integer equal to the number of anomalous behavior creating combinations
    feature_set: array of arrays outer dimension must be the same length as groups if random_features = False
    thresholds: Array containing threshold information for each feature in feature set. Each threshold of a given feature is of the form
                [(value, range, or list), operator code] If rand_features = True thresholds will be chosen 
                randomly as well.
                operator codes:
                    # 0 -> equal to
                    # 1 -> greater than or equal to
                    # 2 -> less than or equal to
                    # 3 -> inclusive range
                    # 4 -> list of discreet values
    anomaly_types: An array of values of 1, 2, or 3 that indicate the "anomaly type" of each of the groups
                anomaly types:
                    # 1 -> slow
                    # 2 -> very slow
                    # 3 -> error                
    feature_set: An array of integers to draw from (with replacement) for choosing the number of features to use in each "group"
                     when random_features = True (if random_features = False this parameter is unused)
    random_groups: Boolean that determines whether or not the number of groups to use will be chosen randomly 
                    (if this is set to true everything well be choosen randomly)
    random_anomalies: Boolean that determines whether or not each groups anamoly type will be chosen randomly
    random_features: Boolean that determines whether or not features and the number of features in each group 
                     will be chosen randomly
    random_thresholds: Boolean that detremines whether or not each groups threholds will be chosen random
                     (automatically random if random_features or random_groups are True)
    random_state: Seed for Random values
    """
    # reset target column to 0

    import numpy as np
    data['anomalous'] = 0

    all_features = ['Age', 'Experience', 'Income', 'CCAvg', 'Mortgage', 'ZIP.Code', 'Family', 'Education',
                    'CD.Account', 'CreditCard', 'Personal.Loan', 'Securities.Account', 'Online']
    continuous_features = ['Age', 'Experience', 'Income', 'CCAvg', 'Mortgage']
    qualitative_m_features = ['ZIP.Code', 'Family', 'Education']
    qualitative_binary_features = ['CD.Account', 'CreditCard', 'Personal.Loan',
                                   'Securities.Account', 'Online']
    np.random.seed(random_state)
    if (random_features or random_groups):
        # generate random number of groups
        if (random_groups):
            groups = np.random.choice([1, 2, 3, 4, 5])
        # generate random feature set
        set_sizes = np.random.choice(feature_set_size, size=groups)
        feature_set = [np.random.choice(all_features, size=x, replace=False) for x in set_sizes]

    # generate random thresholds
    if (random_features or random_groups or random_thresholds):
        thresholds = []
        anomaly_types = []
        for f_set in feature_set:
            t_set = []
            # get range tuples
            for feature in f_set:
                # get anomaly type
                at = np.random.choice([1, 2, 3])
                if (feature in continuous_features):
                    # get operator codes
                    oc = np.random.choice([1, 2, 3])
                    if (oc == 1):
                        quantile = np.random.choice([.5, .75])
                        value = sorted(data[feature])[int(np.floor(quantile * data.shape[0]))]
                    elif (oc == 2):
                        quantile = np.random.choice([.25, .5])
                        value = sorted(data[feature])[int(np.floor(quantile * data.shape[0]))]
                    else:
                        quantile = np.random.choice([0, .25, .75])
                        tuple_1 = sorted(data[feature])[int(np.floor(quantile * data.shape[0]))]
                        tuple_2 = sorted(data[feature])[int(np.floor((quantile + .25) * data.shape[0]))]
                        value = (tuple_1, tuple_2)
                    t_set.append([value, oc])
                elif (feature in qualitative_m_features):
                    # get operator codes
                    oc = np.random.choice([0, 4])
                    if (oc == 0):
                        value = np.random.choice(data[feature])
                    elif (oc == 4):
                        if (feature == 'ZIP.Code'):
                            zip_codes = np.random.choice([x for x in range(1, 6)])
                            value = np.random.choice(np.unique(data[feature]), size=zip_codes, replace=False)
                        elif (feature == 'Family'):
                            family_size = np.random.choice([2, 3])
                            value = np.random.choice(np.unique(data[feature]), size=family_size, replace=False)
                        else:
                            value = np.random.choice(np.unique(data[feature]), size=2, replace=False)
                    t_set.append([value, oc])
                else:
                    t_set.append([1, 0])
            thresholds.append(t_set)

    # build the data frame
    if (random_anomalies or random_groups):
        anomaly_types = np.random.choice([1, 2, 3], size=groups)
    for i, t_set in enumerate(thresholds):
        condition = True
        for j, t_hold in enumerate(t_set):
            if (t_hold[1] == 0):
                condition = condition & (data[feature_set[i][j]] == t_hold[0])
            elif (t_hold[1] == 1):
                condition = condition & (data[feature_set[i][j]] >= t_hold[0])
            elif (t_hold[1] == 2):
                condition = condition & (data[feature_set[i][j]] <= t_hold[0])
            elif (t_hold[1] == 3):
                condition = condition & (
                            (data[feature_set[i][j]] >= t_hold[0][0]) & (data[feature_set[i][j]] <= t_hold[0][1]))
            else:
                condition = condition & (np.isin(data[feature_set[i][j]], t_hold[0]))
            if(anomaly_noise != 0):
                anom_noise = np.random.choice(a=[False, True], size=sum(condition), p=[0.2, 0.8])
                condition[condition] = anom_noise
            data['anomalous'].mask(condition,anomaly_types[i],inplace=True)
            if(normal_noise != 0):
                norm_noise = np.random.choice([1,2,3], size=sum(data['anomalous'] == 0), p=[0.6,0.3,0.1])
                data['anomalous'].mask(data['anomalous']==0, norm_noise, inplace=True)
    # print out what has been made
    anomalies = ['slow', 'very slow', 'error']
    output = "The data frame has the following anomalous groups:"
    for i, t_set in enumerate(thresholds):
        output += "\n" + anomalies[anomaly_types[i] - 1] + " where "
        for j, t_hold in enumerate(t_set):
            output += feature_set[i][j]
            if (t_hold[1] == 0):
                output += " == " + str(t_hold[0])
            elif (t_hold[1] == 1):
                output += " >= " + str(t_hold[0])
            elif (t_hold[1] == 2):
                output += " <= " + str(t_hold[0])
            elif (t_hold[1] == 3):
                output += " is in the inclusive range of " + str(t_hold[0])
            else:
                output += " is in the discrete set of " + str(t_hold[0])
            if (j < len(t_set) - 1):
                output += " and "
    for i in range(1, 4):
        output += "\nThere are " + str(len(data['anomalous'][data['anomalous'] == i])) + " " + anomalies[
            i - 1] + " transactions in this data set"
    return feature_set, thresholds, anomaly_types, output

In [16]:
df = pd.read_excel("/Users/andres/Desktop/CSE498/appdynamics/training_csv/full_set.xlsx")
df = df.drop(['Ô..ID'], axis=1)
df.head()

,Age,Experience,Income,ZIP.Code,Family,CCAvg,Education,Mortgage,Personal.Loan,Securities.Account,CD.Account,Online,CreditCard,responsetime,problem_group,portion,time
1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0,1938.100070,1,0.073529,2020-02-15 00:00:00
2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0,25.165190,0,0.000000,2020-02-15 00:02:52
3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0,2267.739555,4,0.279412,2020-02-15 00:05:45
4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0,18.932080,0,0.000000,2020-02-15 00:08:38
5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1,25.627543,0,0.000000,2020-02-15 00:11:31


In [17]:
data_maker(df)

([['Mortgage', 'Family'], ['Education', 'Income'], ['ZIP.Code']],
 [[[200, 1], [[3, 2], 4]], [[[1, 2], 4], [(50, 100), 3]], [[[91107], 4]]],
 [2, 1, 3],
 'The data frame has the following anomalous groups:\nvery slow where Mortgage >= 200 and Family is in the discrete set of [3, 2]\nslow where Education is in the discrete set of [1, 2] and Income is in the inclusive range of (50, 100)\nerror where ZIP.Code is in the discrete set of [91107]\nThere are 803 slow transactions in this data set\nThere are 150 very slow transactions in this data set\nThere are 47 error transactions in this data set')

In [18]:
df

,Age,Experience,Income,ZIP.Code,Family,CCAvg,Education,Mortgage,Personal.Loan,Securities.Account,CD.Account,Online,CreditCard,responsetime,problem_group,portion,time,anomalous
1,25,1,49,91107,4,1.60,1,0,0,1,0,0,0,1938.100070,1,0.073529,2020-02-15 00:00:00,1
2,45,19,34,90089,3,1.50,1,0,0,1,0,0,0,25.165190,0,0.000000,2020-02-15 00:02:52,1
3,39,15,11,94720,1,1.00,1,0,0,0,0,0,0,2267.739555,4,0.279412,2020-02-15 00:05:45,1
4,35,9,100,94112,1,2.70,2,0,0,0,0,0,0,18.932080,0,0.000000,2020-02-15 00:08:38,1
5,35,8,45,91330,4,1.00,2,0,0,0,0,0,1,25.627543,0,0.000000,2020-02-15 00:11:31,1
6,37,13,29,92121,4,0.40,2,155,0,0,0,1,0,1373.889605,5,0.441176,2020-02-15 00:14:24,2
7,53,27,72,91711,2,1.50,2,0,0,0,0,1,0,26.838841,0,0.000000,2020-02-15 00:17:16,1
8,50,24,22,93943,1,0.30,3,0,0,0,0,0,1,22.588337,0,0.000000,2020-02-15 00:20:09,3
9,35,10,81,90089,3,0.60,2,104,0,0,0,1,0,26.174493,0,0.000000,2020-02-15 00:23:02,1
10,34,9,180,93023,1,8.90,3,0,1,0,0,0,0,25.814199,0,0.000000,2020-02-15 00:25:55,1


In [ ]:
fam_3_edu_2_CCAvg_4_group.xlsx

In [19]:
df = pd.read_excel("/Users/andres/Desktop/CSE498/appdynamics/training_csv/full_set.xlsx")
df = df.drop(['Ô..ID'], axis=1)
df.head()

,Ô..ID,Age,Experience,Income,ZIP.Code,Family,CCAvg,Education,Mortgage,Personal.Loan,Securities.Account,CD.Account,Online,CreditCard,responsetime,problem_group,portion,time
1,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0,1938.100070,1,0.073529,2020-02-15 00:00:00
2,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0,25.165190,0,0.000000,2020-02-15 00:02:52
3,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0,2267.739555,4,0.279412,2020-02-15 00:05:45
4,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0,18.932080,0,0.000000,2020-02-15 00:08:38
5,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1,25.627543,0,0.000000,2020-02-15 00:11:31


1. split all examples into 5 groups. assign them a group. 
2. for every group, tinker with all the features, within a mean.
3. Features that are binary, also, dealth with accordingly.

In [23]:
df = pd.read_excel("/Users/andres/Desktop/CSE498/appdynamics/training_csv/fam_3_edu_2_CCAvg_4_group.xlsx")
df.rename(columns = {'Ô..ID':'id'}, inplace = True) 
df.head()

,id,Age,Experience,Income,ZIP.Code,Family,CCAvg,Education,Mortgage,Personal.Loan,Securities.Account,CD.Account,Online,CreditCard,responsetime,problem_group,portion,time
1,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0,1938.100070,6,0.073529,2020-02-15 00:00:00
2,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0,25.165190,0,0.000000,2020-02-15 00:02:52
3,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0,2267.739555,6,0.279412,2020-02-15 00:05:45
4,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0,18.932080,0,0.000000,2020-02-15 00:08:38
5,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1,25.627543,0,0.000000,2020-02-15 00:11:31
